## Compute the normalization statistics for the GraphCast code

In [1]:
import xarray as xr 
import numpy as np
from glob import glob

import random 
import os

import sys, os 
sys.path.insert(0, os.path.dirname(os.path.dirname(os.getcwd())))

from wofscast.data_generator import load_wofscast_data
from wofscast import my_graphcast as graphcast
import dask 

# Save to NetCDF files
save_path = '/work/mflora/wofs-cast-data/normalization_stats/'

In [2]:
%%time
in_path = '/work/mflora/wofs-cast-data/train_datasets'
n_samples = 1024

with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    inputs = xr.open_dataset(os.path.join(in_path, 'train_inputs.nc'), chunks={})
    forcings = xr.open_dataset(os.path.join(in_path, 'train_forcings.nc'), chunks={})

    full_dataset = xr.merge([inputs, forcings])
    
    # Randomly select batches without replacement
    batch_indices = np.random.choice(full_dataset.batch.size, size=n_samples, replace=False)

    # Select the random batches
    subset_dataset = full_dataset.isel(batch=batch_indices)

    # Setup computations but do not execute them yet
    mean_by_level = subset_dataset.mean(dim=['time', 'lat', 'lon', 'batch'])
    stddev_by_level = subset_dataset .std(dim=['time', 'lat', 'lon', 'batch'])
    
    time_diffs = subset_dataset.diff(dim='time')
    diffs_stddev_by_level = time_diffs.std(dim=['time', 'lat', 'lon', 'batch'])

    mean_by_level, stddev_by_level, diffs_stddev_by_level = dask.compute(mean_by_level, 
                                                                         stddev_by_level, 
                                                                         diffs_stddev_by_level)

    mean_by_level.to_netcdf(os.path.join(save_path, 'mean_by_level.nc'))
    stddev_by_level.to_netcdf(os.path.join(save_path, 'stddev_by_level.nc'))
    diffs_stddev_by_level.to_netcdf(os.path.join(save_path, 'diffs_stddev_by_level.nc'))

    inputs.close()
    forcings.close()

    mean_by_level.close()
    stddev_by_level.close()
    diffs_stddev_by_level.close()
    

/work/mflora/miniconda3/envs/wofs-cast/lib/python3.10/site-packages/dask/array/numpy_compat.py:53: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)


CPU times: user 4min 43s, sys: 1min 9s, total: 5min 52s
Wall time: 6min 43s


In [3]:
import gc
gc.collect()

518